In [ ]:
# !git clone https://github.com/ankitsunil530/blood-cell-classification.git
# %cd blood-cell-classification

fatal: destination path 'blood-cell-classification' already exists and is not an empty directory.
/content/blood-cell-classification/blood-cell-classification


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# **Blood Cell Classification**

## Data loading and preprocessing

### Subtask:
Load the blood cell dataset and preprocess the images for use with the vision transformer and Performer models. This will involve resizing, normalization, and splitting the data into training, validation, and test sets.


In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [7]:
pip install opencv-python


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.8/39.0 MB 1.8 MB/s eta 0:00:22
   - -------------------------------------- 1.3/39.0 MB 1.8 MB/s eta 0:00:21
   - -------------------------------------- 1.6/39.0 MB 1.5 MB/s eta 0:00:25
   -- ------------------------------------- 2.4/39.0 MB 2.0 MB/s eta 0:00:19
   --- ------------------------------------ 3.1/39.0 MB 2.3 MB/s eta 0:00:16
   ---- ----------------------------------- 3.9/39.0 MB 2.5 MB/s eta 0:00:15
   ---- ----------------------------------- 4.7/39.0 MB 2.6 MB/s eta 0:00:14
   ----- ---------------------------------- 5.2/39.0 MB 2.7 MB/s eta 0:00:13
   ------ -------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.5.0 requires numpy<2, but you have numpy 2.2.6 which is incompatible.


In [2]:
DATASET_PATH = "D:\\Deep Learning Project\\blood-cell-classification\\data\\raw\\archive (3)"

IMG_HEIGHT = 224
IMG_WIDTH = 224

def load_images_and_labels(dataset_path):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_path))
    class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
                    images.append(img)
                    labels.append(class_to_idx[class_name])

    return np.array(images), np.array(labels), class_names


In [3]:
for root, dirs, files in os.walk(DATASET_PATH):
    level = root.replace(DATASET_PATH, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')



archive (3)/
    dataset-master/
        dataset-master/
            Annotations/
            JPEGImages/
    dataset2-master/
        dataset2-master/
            images/
                TEST/
                    EOSINOPHIL/
                    LYMPHOCYTE/
                    MONOCYTE/
                    NEUTROPHIL/
                TEST_SIMPLE/
                    EOSINOPHIL/
                    LYMPHOCYTE/
                    MONOCYTE/
                    NEUTROPHIL/
                TRAIN/
                    EOSINOPHIL/
                    LYMPHOCYTE/
                    MONOCYTE/
                    NEUTROPHIL/


In [14]:
def load_images_and_labels(dataset_path):
    images = []
    labels = []
    # Update base directories to reflect the actual structure
    base_image_dirs = [
        os.path.join('dataset2-master', 'dataset2-master', 'images', 'TRAIN'),
        os.path.join('dataset2-master', 'dataset2-master', 'images', 'TEST'),
        os.path.join('dataset2-master', 'dataset2-master', 'images', 'TEST_SIMPLE')
    ]
    class_names = set()

    # First pass to collect all unique class names
    for base_img_dir_name in base_image_dirs:
        base_img_dir_path = os.path.join(dataset_path, base_img_dir_name)
        if os.path.isdir(base_img_dir_path):
            for class_name in os.listdir(base_img_dir_path):
                class_dir_path = os.path.join(base_img_dir_path, class_name)
                if os.path.isdir(class_dir_path):
                    class_names.add(class_name)

    class_names = sorted(list(class_names))
    class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    print(f"Found classes: {class_names}")

    # Second pass to load images
    for base_img_dir_name in base_image_dirs:
        base_img_dir_path = os.path.join(dataset_path, base_img_dir_name)
        if os.path.isdir(base_img_dir_path):
            for class_name in os.listdir(base_img_dir_path):
                class_dir_path = os.path.join(base_img_dir_path, class_name)
                print(f"Processing directory: {class_dir_path}")
                if os.path.isdir(class_dir_path) and class_name in class_to_idx:
                    for img_name in os.listdir(class_dir_path):
                        img_path = os.path.join(class_dir_path, img_name)
                        if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                            img = cv2.imread(img_path)
                            if img is not None:
                                img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
                                images.append(img)
                                labels.append(class_to_idx[class_name])
                            else:
                                print(f"Could not load image: {img_path}")

    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    return np.array(images), np.array(labels), class_names

images, labels, class_names = load_images_and_labels(DATASET_PATH)

# Split the data - combining images from TRAIN, TEST, TEST_SIMPLE before splitting
# Using a fixed random state for reproducibility
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")
print(f"Class names: {class_names}")


Found classes: ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TRAIN\EOSINOPHIL
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TRAIN\LYMPHOCYTE
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TRAIN\MONOCYTE
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TRAIN\NEUTROPHIL
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TEST\EOSINOPHIL
Processing directory: D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)\dataset2-master\dataset2-master\images\TEST\LYMPHOCYTE
Processing directory: D:\Deep Lear

In [4]:
# train_gan.py
import os
import math
import time
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.utils.data import DataLoader

import mlflow
import mlflow.pytorch

# --------------------
# Config / Hyperparams
# --------------------
DATASET_PATH = r"D:\Deep Learning Project\blood-cell-classification\data\raw\archive (3)"  # set correct path
EXPERIMENT = "Blood-Cell-GAN"
RUN_NAME = "DCGAN-run1"
OUT_DIR = "gan_outputs"
MODEL_DIR = "models"
IMG_SIZE = 128                 # use 128 (faster than 224)
BATCH_SIZE = 64
Z_DIM = 100
NUM_EPOCHS = 100
LR = 2e-4
BETA1 = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAMPLE_EVERY = 5               # epochs
NUM_WORKERS = 4

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# --------------------
# Data preparation
# --------------------
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),  # images normalized to [-1,1]
])

dataset = datasets.ImageFolder(root=DATASET_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

print("Dataset classes:", dataset.classes)
print("Dataset size:", len(dataset))

# --------------------
# Models (DCGAN-style)
# --------------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, z_dim=100, ngf=64, nc=3):
        super().__init__()
        # for 128x128 output, we need more layers
        self.net = nn.Sequential(
            # input z -> (ngf*16) x 4 x 4
            nn.ConvTranspose2d(z_dim, ngf*16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*16),
            nn.ReLU(True),

            # state size ngf*16 x 4 x 4 -> ngf*8 x 8 x 8
            nn.ConvTranspose2d(ngf*16, ngf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            # -> ngf*4 x 16 x 16
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            # -> ngf*2 x 32 x 32
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            # -> ngf x 64 x 64
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # -> nc x 128 x 128
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.net = nn.Sequential(
            # input nc x 128 x 128
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, ndf*16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*16),
            nn.LeakyReLU(0.2, inplace=True),

            # final
            nn.Conv2d(ndf*16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1, 1).squeeze(1)

# instantiate
netG = Generator(Z_DIM).to(DEVICE)
netD = Discriminator().to(DEVICE)
netG.apply(weights_init)
netD.apply(weights_init)

# Loss & optimizers
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(BETA1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(BETA1, 0.999))

fixed_noise = torch.randn(64, Z_DIM, 1, 1, device=DEVICE)  # for sample grid

# --------------------
# MLflow setup
# --------------------
mlflow.set_experiment(EXPERIMENT)
mlflow.start_run(run_name=RUN_NAME)
mlflow.log_params({
    "img_size": IMG_SIZE, "batch_size": BATCH_SIZE, "z_dim": Z_DIM,
    "lr": LR, "beta1": BETA1, "epochs": NUM_EPOCHS, "device": str(DEVICE)
})

# --------------------
# Training loop
# --------------------
real_label_val = 1.0
fake_label_val = 0.0

iters = 0
for epoch in range(1, NUM_EPOCHS+1):
    epoch_start = time.time()
    running_D_loss = 0.0
    running_G_loss = 0.0
    for i, (data, _) in enumerate(dataloader):
        netD.zero_grad()
        real_cpu = data.to(DEVICE)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label_val, dtype=torch.float, device=DEVICE)

        # Train D on real
        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # Train D on fake
        noise = torch.randn(b_size, Z_DIM, 1, 1, device=DEVICE)
        fake = netG(noise)
        label.fill_(fake_label_val)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        # Train G
        netG.zero_grad()
        label.fill_(real_label_val)  # want generator to produce real labels
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        running_D_loss += errD.item()
        running_G_loss += errG.item()

        # log per-batch occasionally (avoid too many logs)
        if iters % 100 == 0:
            mlflow.log_metric("D_loss_batch", errD.item(), step=iters)
            mlflow.log_metric("G_loss_batch", errG.item(), step=iters)

        iters += 1

    # epoch stats
    avg_D = running_D_loss / len(dataloader)
    avg_G = running_G_loss / len(dataloader)
    epoch_time = time.time() - epoch_start
    print(f"Epoch [{epoch}/{NUM_EPOCHS}]  D_loss: {avg_D:.4f}  G_loss: {avg_G:.4f}  time: {epoch_time:.1f}s")

    mlflow.log_metric("D_loss_epoch", avg_D, step=epoch)
    mlflow.log_metric("G_loss_epoch", avg_G, step=epoch)

    # save models & samples periodically
    if epoch % SAMPLE_EVERY == 0 or epoch == 1:
        # save checkpoints
        torch.save(netG.state_dict(), os.path.join(MODEL_DIR, f"netG_epoch{epoch}.pth"))
        torch.save(netD.state_dict(), os.path.join(MODEL_DIR, f"netD_epoch{epoch}.pth"))

        # save as mlflow artifacts
        mlflow.log_artifact(os.path.join(MODEL_DIR, f"netG_epoch{epoch}.pth"))
        mlflow.log_artifact(os.path.join(MODEL_DIR, f"netD_epoch{epoch}.pth"))

        # generate sample grid
        with torch.no_grad():
            fake_samples = netG(fixed_noise).detach().cpu()
        grid = utils.make_grid(fake_samples, padding=2, normalize=True)
        sample_path = os.path.join(OUT_DIR, f"sample_epoch{epoch}.png")
        utils.save_image(grid, sample_path)
        mlflow.log_artifact(sample_path)
        print(f"Saved sample and checkpoints for epoch {epoch}")

# end run
mlflow.end_run()
print("Training finished!")


Dataset classes: ['dataset-master', 'dataset2-master']
Dataset size: 12881


KeyboardInterrupt: 

In [5]:
import torch

# Check GPU availability
print("CUDA Available:", torch.cuda.is_available())

# Get GPU name
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Total GPUs:", torch.cuda.device_count())
    print("Current Device:", torch.cuda.current_device())
else:
    print("No GPU detected. Running on CPU.")


CUDA Available: False
No GPU detected. Running on CPU.


In [19]:
import mlflow
mlflow.end_run()


## Data loading and preprocessing

### Subtask:
Load the blood cell dataset and preprocess the images for use with the vision transformer and Performer models. This will involve resizing, normalization, and splitting the data into training, validation, and test sets.


In [8]:
class BloodCellDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        """
        Args:
            images (numpy array): Array of image data (H, W, C).
            labels (numpy array): Array of corresponding labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)

        return image, label


In [9]:
# Define transformations
# Using common mean and std dev for ImageNet as a starting point
# A more accurate approach would be to calculate these from the dataset
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.ToPILImage(), # Convert numpy array to PIL Image for transforms
    transforms.RandomResizedCrop(IMG_HEIGHT),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

val_test_transform = transforms.Compose([
    transforms.ToPILImage(), # Convert numpy array to PIL Image for transforms
    transforms.Resize(IMG_HEIGHT),
    transforms.CenterCrop(IMG_HEIGHT),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Create Dataset instances
train_dataset = BloodCellDataset(X_train, y_train, transform=train_transform)
val_dataset = BloodCellDataset(X_val, y_val, transform=val_test_transform)
test_dataset = BloodCellDataset(X_test, y_test, transform=val_test_transform)

# Create DataLoader instances
batch_size = 32 # Using the already defined batch_size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of training batches: {len(train_dataloader)}")
print(f"Number of validation batches: {len(val_dataloader)}")
print(f"Number of test batches: {len(test_dataloader)}")


Number of training batches: 313
Number of validation batches: 40
Number of test batches: 40


## Vision transformer model implementation

### Subtask:
Implement the Vision Transformer model for blood cell classification.


In [10]:
import torch.nn as nn
from torchvision.models import vit_b_16, ViT_B_16_Weights

In [28]:
class BloodCellViT(nn.Module):
    def __init__(self, num_classes=4):
        super(BloodCellViT, self).__init__()
        # Load pre-trained ViT-Base/16
        weights = ViT_B_16_Weights.DEFAULT
        self.vit = vit_b_16(weights=None)
        num_ftrs = self.vit.heads.head.in_features
        self.vit.heads.head = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.vit(x)


num_classes = 4 # Based on the previous data loading step
model = BloodCellViT(num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model initialized and moved to {device}")
print(model)

Model initialized and moved to cuda
BloodCellViT(
  (vit): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): 

## Performer model implementation

### Subtask:
Implement the Performer model for blood cell classification.


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
def linear_attention(q, k, v):
    attention_weights = torch.matmul(q, k.transpose(-2, -1))
    attention_weights = F.softmax(attention_weights, dim=-1)
    output = torch.matmul(attention_weights, v)
    return output

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x

class PerformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4., drop_rate=0., attn_drop_rate=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=attn_drop_rate, batch_first=True) # Using standard MultiheadAttention for now, would replace with efficient attention
        self.norm2 = nn.LayerNorm(embed_dim)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(drop_rate),
            nn.Linear(mlp_hidden_dim, embed_dim),
            nn.Dropout(drop_rate)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

class PerformerModel(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768, num_layers=12, num_heads=12, mlp_ratio=4., num_classes=4, drop_rate=0., attn_drop_rate=0.):
        super().__init__()
        self.num_classes = num_classes
        self.embed_dim = embed_dim

        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        self.blocks = nn.ModuleList([
            PerformerBlock(embed_dim, num_heads, mlp_ratio, drop_rate, attn_drop_rate)
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        # Initialize weights
        nn.init.trunc_normal_(self.pos_embed, std=.02)
        nn.init.trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        return x[:, 0]

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x


In [2]:

img_size = IMG_HEIGHT
patch_size = 16
in_channels = 3
embed_dim = 768
num_layers = 12
num_heads = 12
mlp_ratio = 4
num_classes = num_classes

performer_model = PerformerModel(
    img_size=img_size,
    patch_size=patch_size,
    in_channels=in_channels,
    embed_dim=embed_dim,
    num_layers=num_layers,
    num_heads=num_heads,
    mlp_ratio=mlp_ratio,
    num_classes=num_classes
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
performer_model.to(device)

print(f"Performer Model initialized and moved to {device}")
print(performer_model)

NameError: name 'IMG_HEIGHT' is not defined

## Model training

### Subtask:
Train both the Vision Transformer and Performer models on the training data.


# Newly Initialized ViT Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vit_b_16, ViT_B_16_Weights
from tqdm import tqdm

def train_vit(model, train_loader, val_loader, criterion, optimizer, device, epochs=5):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] → Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

        val_acc = test_vit(model, val_loader, device)
        print(f"Validation Accuracy: {val_acc:.2f}%")

    return model

def test_vit(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 4
criterion = nn.CrossEntropyLoss()

vit_model = BloodCellViT(num_classes)
optimizer = optim.AdamW(vit_model.parameters(), lr=1e-4)

vit_model = train_vit(vit_model, train_dataloader, val_dataloader, criterion, optimizer, device, epochs=5)
vit_test_acc = test_vit(vit_model, test_dataloader, device)
print(f"\n✅ Final ViT Test Accuracy: {vit_test_acc:.2f}%")


Epoch 1/5: 100%|██████████| 313/313 [06:24<00:00,  1.23s/it]


Epoch [1/5] → Loss: 1.4277, Train Acc: 28.13%
Validation Accuracy: 56.12%


Epoch 2/5: 100%|██████████| 313/313 [06:24<00:00,  1.23s/it]


Epoch [2/5] → Loss: 0.8736, Train Acc: 62.50%
Validation Accuracy: 83.21%


Epoch 3/5: 100%|██████████| 313/313 [06:24<00:00,  1.23s/it]


Epoch [3/5] → Loss: 0.6239, Train Acc: 72.64%
Validation Accuracy: 84.73%


Epoch 4/5: 100%|██████████| 313/313 [06:25<00:00,  1.23s/it]


Epoch [4/5] → Loss: 0.7036, Train Acc: 69.95%
Validation Accuracy: 86.89%


Epoch 5/5: 100%|██████████| 313/313 [06:28<00:00,  1.24s/it]


Epoch [5/5] → Loss: 0.5512, Train Acc: 75.38%
Validation Accuracy: 86.49%

✅ Final ViT Test Accuracy: 84.74%


In [1]:
import torch
import mlflow
import mlflow.pytorch

# Assume 'performer_model' is your trained model object
performer_model.eval()  # Set to evaluation mode

# 🔹 1. Save model to disk
torch.save(performer_model.state_dict(), "models/performer_model.pth")
print("✅ Performer model saved to disk as performer_model.pth")

# 🔹 2. Log model to MLflow
mlflow.set_tracking_uri("file:///D:/Blood Cell Classifiaction/blood-cell-classification/mlruns")
mlflow.set_experiment("Blood-Cell-Classification-Results")

with mlflow.start_run(run_name="Performer-Model-Logged"):
    mlflow.pytorch.log_model(performer_model, artifact_path="performer_model")
    
    # Log parameters
    mlflow.log_param("model_type", "Performer (Efficient Transformer)")
    mlflow.log_param("epochs", 5)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("num_layers", 12)
    mlflow.log_param("embed_dim", 768)
    mlflow.log_param("num_heads", 12)
    
    # 🔹 3. Log metrics per epoch
    epoch_metrics = [
        {"loss":1.4277, "train_acc":28.13, "val_acc":56.12},
        {"loss":0.8736, "train_acc":62.50, "val_acc":83.21},
        {"loss":0.6239, "train_acc":72.64, "val_acc":84.73},
        {"loss":0.7036, "train_acc":69.95, "val_acc":86.89},
        {"loss":0.5512, "train_acc":75.38, "val_acc":86.49}
    ]
    
    for i, metrics in enumerate(epoch_metrics, start=1):
        mlflow.log_metric("loss", metrics["loss"], step=i)
        mlflow.log_metric("train_acc", metrics["train_acc"], step=i)
        mlflow.log_metric("val_acc", metrics["val_acc"], step=i)
    
    # Final test accuracy
    mlflow.log_metric("final_test_accuracy", 84.74)

print("✅ Performer model logged to MLflow successfully!")


NameError: name 'performer_model' is not defined

# **PreTrained ViT Model**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vit_b_16, ViT_B_16_Weights
from tqdm import tqdm
import mlflow
import mlflow.pytorch

class BloodCellViT(nn.Module):
    def __init__(self, num_classes=4):
        super(BloodCellViT, self).__init__()
        # Pretrained ViT model
        weights = ViT_B_16_Weights.DEFAULT
        self.vit = vit_b_16(weights=weights)
        num_ftrs = self.vit.heads.head.in_features
        self.vit.heads.head = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.vit(x)

def test_vit(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

def train_vit(model, train_loader, val_loader, criterion, optimizer, device, epochs=5):
    model.to(device)
    
    # START MLFLOW RUN
    with mlflow.start_run(run_name=f"ViT-PreTrained-{epochs}epochs"):
        
        # Log hyperparameters
        mlflow.log_param("model_type", "Vision Transformer (PreTrained)")
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("optimizer", "AdamW")
        mlflow.log_param("batch_size", train_loader.batch_size)
        mlflow.log_param("device", str(device))
        mlflow.log_param("learning_rate", 1e-4)
        
        print(f"\n🚀 Starting ViT Training with MLflow tracking...")
        print(f"   Run name: ViT-PreTrained-{epochs}epochs")
        
        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0, 0, 0

            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [TRAIN]"):
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_acc = 100 * correct / total
            train_loss = running_loss / len(train_loader)
            
            # Validation
            val_acc = test_vit(model, val_loader, device)
            
            print(f"Epoch [{epoch+1}/{epochs}] → Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
            
            # Log metrics to MLflow
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_acc, step=epoch)
            mlflow.log_metric("val_accuracy", val_acc, step=epoch)
        
        # SAVE AND LOG THE MODEL
        print(f"\n💾 Saving model to MLflow...")
        mlflow.pytorch.log_model(model, artifact_path="vit_model")
        
        print(f"✅ MLflow run completed! View at http://localhost:5000")
        
    return model

# Initialize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 4
criterion = nn.CrossEntropyLoss()

print(f"\n📊 Model Setup:")
print(f"   Device: {device}")
print(f"   Classes: {num_classes}")

vit_model = BloodCellViT(num_classes)
optimizer = optim.AdamW(vit_model.parameters(), lr=1e-4)

# Train with MLflow tracking
vit_model = train_vit(vit_model, train_dataloader, val_dataloader, criterion, optimizer, device, epochs=5)

# Final test
vit_test_acc = test_vit(vit_model, test_dataloader, device)
print(f"\n✅ Final ViT Test Accuracy: {vit_test_acc:.2f}%")


Epoch 1/5: 100%|██████████| 313/313 [06:29<00:00,  1.24s/it]


Epoch [1/5] → Loss: 0.5570, Train Acc: 75.55%
Validation Accuracy: 91.93%


Epoch 2/5: 100%|██████████| 313/313 [06:27<00:00,  1.24s/it]


Epoch [2/5] → Loss: 0.3417, Train Acc: 85.24%
Validation Accuracy: 96.56%


Epoch 3/5: 100%|██████████| 313/313 [06:28<00:00,  1.24s/it]


Epoch [3/5] → Loss: 0.3072, Train Acc: 87.06%
Validation Accuracy: 96.24%


Epoch 4/5: 100%|██████████| 313/313 [06:27<00:00,  1.24s/it]


Epoch [4/5] → Loss: 0.2820, Train Acc: 87.87%
Validation Accuracy: 96.64%


Epoch 5/5: 100%|██████████| 313/313 [06:28<00:00,  1.24s/it]


Epoch [5/5] → Loss: 0.2511, Train Acc: 89.58%
Validation Accuracy: 97.04%

✅ Final ViT Test Accuracy: 97.12%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import mlflow
import mlflow.pytorch

def test_performer(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

def train_performer(model, train_loader, val_loader, criterion, optimizer, device, epochs=5):
    model.to(device)
    
    # START MLFLOW RUN
    with mlflow.start_run(run_name=f"Performer-{epochs}epochs"):
        
        # Log hyperparameters
        mlflow.log_param("model_type", "Performer")
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("optimizer", "AdamW")
        mlflow.log_param("batch_size", train_loader.batch_size)
        mlflow.log_param("device", str(device))
        mlflow.log_param("learning_rate", 1e-4)
        mlflow.log_param("num_layers", 12)
        mlflow.log_param("embed_dim", 768)
        
        print(f"\n🚀 Starting Performer Training with MLflow tracking...")
        print(f"   Run name: Performer-{epochs}epochs")
        
        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0, 0, 0

            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [TRAIN]"):
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_acc = 100 * correct / total
            train_loss = running_loss / len(train_loader)
            
            # Validation
            val_acc = test_performer(model, val_loader, device)
            
            print(f"Epoch [{epoch+1}/{epochs}] → Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
            
            # Log metrics to MLflow
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_acc, step=epoch)
            mlflow.log_metric("val_accuracy", val_acc, step=epoch)
        
        # SAVE AND LOG THE MODEL
        print(f"\n💾 Saving model to MLflow...")
        mlflow.pytorch.log_model(model, artifact_path="performer_model")
        
        print(f"✅ MLflow run completed! View at http://localhost:5000")
        
    return model

# Initialize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(performer_model.parameters(), lr=1e-4)

print(f"\n📊 Performer Model Setup:")
print(f"   Device: {device}")

# Train with MLflow tracking
performer_model = train_performer(performer_model, train_dataloader, val_dataloader, criterion, optimizer, device, epochs=5)

# Final test
performer_test_acc = test_performer(performer_model, test_dataloader, device)
print(f"\n✅ Final Performer Test Accuracy: {performer_test_acc:.2f}%")


Epoch 1/5: 100%|██████████| 313/313 [06:19<00:00,  1.21s/it]


Epoch [1/5] → Loss: 1.4087, Train Acc: 30.07%
Validation Accuracy: 46.52%


Epoch 2/5: 100%|██████████| 313/313 [06:19<00:00,  1.21s/it]


Epoch [2/5] → Loss: 0.9530, Train Acc: 57.67%
Validation Accuracy: 77.30%


Epoch 3/5: 100%|██████████| 313/313 [06:19<00:00,  1.21s/it]


Epoch [3/5] → Loss: 0.6807, Train Acc: 70.94%
Validation Accuracy: 83.37%


Epoch 4/5: 100%|██████████| 313/313 [06:19<00:00,  1.21s/it]


Epoch [4/5] → Loss: 0.5925, Train Acc: 74.37%
Validation Accuracy: 87.77%


Epoch 5/5: 100%|██████████| 313/313 [06:21<00:00,  1.22s/it]


Epoch [5/5] → Loss: 0.5276, Train Acc: 76.98%
Validation Accuracy: 88.09%

✅ Final Performer Test Accuracy: 88.90%


In [2]:
import mlflow

# Test logging to verify connection
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
with mlflow.start_run():
    print("✓ Successfully connected to MLflow!")

MLflow Tracking URI: file:///d:/Blood%20Cell%20Classifiaction/blood-cell-classification/notebooks/mlruns
✓ Successfully connected to MLflow!


In [4]:
import mlflow

mlflow.set_experiment("MLflow Quickstart")

2025/11/30 00:55:26 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/Blood%20Cell%20Classifiaction/blood-cell-classification/notebooks/mlruns/386924970153409238', creation_time=1764444326364, experiment_id='386924970153409238', last_update_time=1764444326364, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

In [5]:
!pip install --upgrade protobuf==3.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
databricks-sdk 0.73.0 requires protobuf!=5.26.*,!=5.27.*,!=5.28.*,!=5.29.0,!=5.29.1,!=5.29.2,!=5.29.3,!=5.29.4,!=6.30.0,!=6.30.1,!=6.31.0,<7.0,>=4.25.8, but you have protobuf 3.20.3 which is incompatible.
You should consider upgrading via the 'C:\Users\Sunil Kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


In [7]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///d:/Blood%20Cell%20Classifiaction/blood-cell-classification/notebooks/mlruns


In [9]:
import mlflow
import mlflow.pytorch

# Set MLflow tracking URI
mlflow.set_tracking_uri("file:///D:/Blood Cell Classifiaction/blood-cell-classification/mlruns")

print("\n" + "="*80)
print("📊 LOGGING TRAINED MODEL RESULTS TO MLFLOW")
print("="*80)

mlflow.set_experiment("Blood-Cell-Classification-Results")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1️⃣ CUSTOM VIT MODEL RESULTS
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n🚀 Logging Custom ViT Model Results...")
with mlflow.start_run(run_name="Custom-ViT-Model"):
    mlflow.log_param("model_type", "Custom Vision Transformer")
    mlflow.log_param("epochs", 5)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("batch_size", 32)

    # Epoch-wise metrics
    loss_list = [1.4277, 0.8736, 0.6239, 0.7036, 0.5512]
    train_acc_list = [28.13, 62.50, 72.64, 69.95, 75.38]
    val_acc_list = [56.12, 83.21, 84.73, 86.89, 86.49]

    for epoch in range(5):
        mlflow.log_metric("loss", loss_list[epoch], step=epoch+1)
        mlflow.log_metric("train_accuracy", train_acc_list[epoch], step=epoch+1)
        mlflow.log_metric("val_accuracy", val_acc_list[epoch], step=epoch+1)

    mlflow.log_metric("final_test_accuracy", 84.74)
    print("✅ Custom ViT results logged!")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2️⃣ PRETRAINED VIT MODEL RESULTS
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n🚀 Logging PreTrained ViT Model Results...")
with mlflow.start_run(run_name="PreTrained-ViT-Model"):
    mlflow.log_param("model_type", "Vision Transformer (PreTrained ViT-B/16)")
    mlflow.log_param("epochs", 5)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("pretrained_weights", "ViT_B_16_Weights.DEFAULT")

    # Epoch-wise metrics
    loss_list = [0.5570, 0.3417, 0.3072, 0.2820, 0.2511]
    train_acc_list = [75.55, 85.24, 87.06, 87.87, 89.58]
    val_acc_list = [91.93, 96.56, 96.24, 96.64, 97.04]

    for epoch in range(5):
        mlflow.log_metric("loss", loss_list[epoch], step=epoch+1)
        mlflow.log_metric("train_accuracy", train_acc_list[epoch], step=epoch+1)
        mlflow.log_metric("val_accuracy", val_acc_list[epoch], step=epoch+1)

    mlflow.log_metric("final_test_accuracy", 97.12)
    print("✅ PreTrained ViT results logged!")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 3️⃣ PERFORMER MODEL RESULTS
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n🚀 Logging Performer Model Results...")
with mlflow.start_run(run_name="Performer-Model"):
    mlflow.log_param("model_type", "Performer (Efficient Transformer)")
    mlflow.log_param("epochs", 5)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("learning_rate", 1e-4)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("num_layers", 12)
    mlflow.log_param("embed_dim", 768)
    mlflow.log_param("num_heads", 12)

    # Epoch-wise metrics
    loss_list = [1.4087, 0.9530, 0.6807, 0.5925, 0.5276]
    train_acc_list = [30.07, 57.67, 70.94, 74.37, 76.98]
    val_acc_list = [46.52, 77.30, 83.37, 87.77, 88.09]

    for epoch in range(5):
        mlflow.log_metric("loss", loss_list[epoch], step=epoch+1)
        mlflow.log_metric("train_accuracy", train_acc_list[epoch], step=epoch+1)
        mlflow.log_metric("val_accuracy", val_acc_list[epoch], step=epoch+1)

    mlflow.log_metric("final_test_accuracy", 88.90)
    print("✅ Performer results logged!")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# SUMMARY
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*80)
print("✅ ALL RESULTS LOGGED TO MLFLOW!")
print("="*80)
print("\n📊 SUMMARY:")
print(f"  • Custom ViT:        Final Test Accuracy = 84.74%")
print(f"  • PreTrained ViT:    Final Test Accuracy = 97.12% ⭐ BEST")
print(f"  • Performer Model:   Final Test Accuracy = 88.90%")
print("\n🌐 Open MLflow Dashboard:")
print("   👉 http://localhost:5000")
print("\n📁 Experiment: Blood-Cell-Classification-Results")
print("="*80 + "\n")



📊 LOGGING TRAINED MODEL RESULTS TO MLFLOW

🚀 Logging Custom ViT Model Results...
✅ Custom ViT results logged!

🚀 Logging PreTrained ViT Model Results...
✅ PreTrained ViT results logged!

🚀 Logging Performer Model Results...
✅ Performer results logged!

✅ ALL RESULTS LOGGED TO MLFLOW!

📊 SUMMARY:
  • Custom ViT:        Final Test Accuracy = 84.74%
  • PreTrained ViT:    Final Test Accuracy = 97.12% ⭐ BEST
  • Performer Model:   Final Test Accuracy = 88.90%

🌐 Open MLflow Dashboard:
   👉 http://localhost:5000

📁 Experiment: Blood-Cell-Classification-Results



In [3]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///D:/Blood Cell Classifiaction/blood-cell-classification/mlruns


In [11]:
import mlflow
import mlflow.pytorch

mlflow.set_tracking_uri("file:///D:/Blood Cell Classifiaction/blood-cell-classification/mlruns")
mlflow.set_experiment("Blood-Cell-Classification-Results")

def log_model_results(run_name, model_type, epochs, metrics, params=None):
    """Log metrics and parameters for a model to MLflow."""
    if params is None:
        params = {}
    
    with mlflow.start_run(run_name=run_name):
        # Log parameters
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("epochs", epochs)
        for key, val in params.items():
            mlflow.log_param(key, val)
        
        # Log metrics per epoch
        for metric_type in ["train_acc", "val_acc", "loss"]:
            if metric_type in metrics:
                for epoch in range(1, epochs + 1):
                    key = f"epoch_{epoch}_{metric_type}"
                    if key in metrics[metric_type]:
                        mlflow.log_metric(metric_type, metrics[metric_type][key], step=epoch)
        
        # Log final test accuracy
        if "final_test_accuracy" in metrics:
            mlflow.log_metric("final_test_accuracy", metrics["final_test_accuracy"])

# Example: Custom ViT
custom_vit_metrics = {
    "train_acc": {
        "epoch_1_train_acc":28.13, "epoch_2_train_acc":62.50,
        "epoch_3_train_acc":72.64, "epoch_4_train_acc":69.95,
        "epoch_5_train_acc":75.38
    },
    "val_acc": {
        "epoch_1_val_acc":56.12, "epoch_2_val_acc":83.21,
        "epoch_3_val_acc":84.73, "epoch_4_val_acc":86.89,
        "epoch_5_val_acc":86.49
    },
    "loss": {
        "epoch_1_loss":1.4277, "epoch_2_loss":0.8736,
        "epoch_3_loss":0.6239, "epoch_4_loss":0.7036,
        "epoch_5_loss":0.5512
    },
    "final_test_accuracy": 84.74
}
custom_params = {"optimizer":"AdamW","learning_rate":1e-4,"batch_size":32}

log_model_results("Custom-ViT-Model", "Custom Vision Transformer", 5, custom_vit_metrics, custom_params)
